<a href="https://colab.research.google.com/github/cakwok/CS6140-Machine-Learning/blob/main/1_1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [263]:
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split  #Split training and testing data

In [264]:
class ClassificationDecisionTree:
    def __init__(self, max_depth=10, min_samples=2):
        self.max_depth = max_depth
        self.min_samples = min_samples
        self.root = None

    def _is_finished(self, depth):
        if (depth >= self.max_depth
            or self.n_class_labels == 1
            or self.n_samples < self.min_samples):
            return True
        return False
    
    def _build_tree(self, X, y, depth=0):
        self.n_samples, self.n_features = X.shape
        self.n_class_labels = len(np.unique(y))

        # stopping criteria
        if self._is_finished(depth):
            most_common_Label = np.argmax(np.bincount(y))
            print("Node(value=most_common_Label)", Node(value=most_common_Label)).value
            return Node(value=most_common_Label)
        
    def fit(self, X, y):
        self.root = self._build_tree(X, y)

    def _entropy(self, y):
        proportions = np.bincount(y) / len(y)
        entropy = -np.sum([p * np.log2(p) for p in proportions if p > 0])
        return entropy

    def _create_split(self, X, thresh):
        left_idx = np.argwhere(X <= thresh).flatten()
        right_idx = np.argwhere(X > thresh).flatten()
        return left_idx, right_idx

    def _information_gain(self, X, y, thresh):
        parent_loss = self._entropy(y)
        left_idx, right_idx = self._create_split(X, thresh)
        n, n_left, n_right = len(y), len(left_idx), len(right_idx)

        if n_left == 0 or n_right == 0: 
            return 0
        
        child_loss = (n_left / n) * self._entropy(y[left_idx]) + (n_right / n) * self._entropy(y[right_idx])
        return parent_loss - child_loss

    def _best_split(self, X, y, features):
        split = {'score':- 1, 'feat': None, 'thresh': None}

        for feat in features:
            X_feat = X[:, feat]                #x[row_index, column_index]
            print ("X_feat", X_feat)
            thresholds = np.unique(X_feat)     #Returns the sorted unique elements of an array.
            print ("thresholds", thresholds)
            for thresh in thresholds:
                score = self._information_gain(X_feat, y, thresh)

                if score > split['score']:
                    split['score'] = score
                    split['feat'] = feat
                    split['thresh'] = thresh

        return split['feat'], split['thresh']
    
    def _build_tree(self, X, y, depth=0):

        self.n_samples, self.n_features = X.shape
        self.n_class_labels = len(np.unique(y))

        # stopping criteria
        if self._is_finished(depth):
            most_common_Label = np.argmax(np.bincount(y))
            return Node(value=most_common_Label)

        # get best split
        rnd_feats = np.random.choice(self.n_features, self.n_features, replace=False)
        print(rnd_feats)
        best_feat, best_thresh = self._best_split(X, y, rnd_feats)
        print (best_feat, best_thresh,"\n\n")

        # grow children recursively
        left_idx, right_idx = self._create_split(X[:, best_feat], best_thresh)
        left_child = self._build_tree(X[left_idx, :], y[left_idx], depth + 1)
        right_child = self._build_tree(X[right_idx, :], y[right_idx], depth + 1)
        return Node(best_feat, best_thresh, left_child, right_child)

    def _traverse_tree(self, x, node):
        if node.is_leaf():
            print ("node.value", node.value)
            return node.value
        
        print ("x", x)
        print ("x[node.feature]", x[node.feature], "node.threshold", node.threshold)
        print ("vars(node)", vars(node))
        print ("node", node)
        print ("node.feature", node.feature)
    
        if x[node.feature] <= node.threshold:
            return self._traverse_tree(x, node.left)
        return self._traverse_tree(x, node.right)
        
    def _print_tree(self, node):
      if node.is_leaf():
          print ("node", node)
          print ("Class", node.value)
          return node.value
      
      print ("node", node) 
      print ("feature", node.feature, "<=", node.threshold )
      print ("node.left", node.left)
      print ("node.right", node.right)
      print ("\n")
      
      print ("left ", end='')
      self._print_tree(node.left)

      print ("right ", end='')
      self._print_tree(node.right)

    def predict(self, X):
        for x in X:
          print("X"); print (X)
          print("x"); print (x)
        predictions = [self._traverse_tree(x, self.root) for x in X]
        return np.array(predictions)
        

In [265]:
class Node:
    def __init__(self, feature=None, threshold=None, left=None, right=None, *, value=None):
        self.feature = feature
        self.threshold = threshold
        self.left = left
        self.right = right
        self.value = value
    
    def is_leaf(self):
        return self.value is not None

In [266]:
def accuracy(y_true, y_pred):
    accuracy = np.sum(y_true == y_pred) / len(y_true)
    return accuracy

In [267]:
dataset = pd.read_csv("/content/drive/My Drive/Colab Notebooks/CS6140 Assignment1/data.csv")


In [ ]:
feature_cols = ['feature1', 'feature2', 'feature3', 'feature4']
X = dataset[feature_cols]                         #Assign all feature columns into variable X
y = dataset['class']                              #Assign all target columns into variable y

X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.2, random_state=1
)

model = ClassificationDecisionTree(max_depth=10)
model.fit(X_train.values , y_train.values)

In [ ]:
print ("X_test", X_test)
y_pred = model.predict(X_test.values)
print ("y_pred", y_pred)
acc = accuracy(y_test, y_pred)
print("Accuracy:", acc)

In [271]:
model._print_tree(model.root)



node <__main__.Node object at 0x7f592ee93390>
feature 2 <= 1.9
node.left <__main__.Node object at 0x7f592eda5d50>
node.right <__main__.Node object at 0x7f593c3ed350>


left node <__main__.Node object at 0x7f592eda5d50>
Class 0
right node <__main__.Node object at 0x7f593c3ed350>
feature 3 <= 1.6
node.left <__main__.Node object at 0x7f592eeb7e90>
node.right <__main__.Node object at 0x7f592eeb7d90>


left node <__main__.Node object at 0x7f592eeb7e90>
feature 2 <= 4.9
node.left <__main__.Node object at 0x7f592edb6d50>
node.right <__main__.Node object at 0x7f592eeb7fd0>


left node <__main__.Node object at 0x7f592edb6d50>
Class 1
right node <__main__.Node object at 0x7f592eeb7fd0>
feature 3 <= 1.5
node.left <__main__.Node object at 0x7f592ed32910>
node.right <__main__.Node object at 0x7f592eeb7f90>


left node <__main__.Node object at 0x7f592ed32910>
Class 2
right node <__main__.Node object at 0x7f592eeb7f90>
Class 1
right node <__main__.Node object at 0x7f592eeb7d90>
Class 2
